Data Generation

In [17]:
import numpy as np
from geomdl import BSpline
from geomdl import utilities
from geomdl import operations
from geomdl import exchange
from geomdl import shapes
from geomdl import construct
from geomdl.visualization import VisPlotly as vis
from geomdl import CPGen
from matplotlib import cm
import random

In [31]:
#Helper Functions

#from https://stackoverflow.com/questions/8487893/generate-all-the-points-on-the-circumference-of-a-circle
def PointsInCircum(r,z,n=15):
    return [[np.cos(2*np.pi/n*x)*r,np.sin(2*np.pi/n*x)*r, float(z)] for x in range(0,n + 1)]

Generate Geometry

In [81]:
#Initialize vertices of bottle

numLevels = 5
numPtsOnLevel = 6
stepBetweenLevels = 1
radius = 2
offset = 1

mouthVerticesInner = [PointsInCircum(3 + offset,0,numPtsOnLevel)]
topVerticesInner = [PointsInCircum(radius + offset,(numLevels + 1) * stepBetweenLevels,numPtsOnLevel)]

print((numPtsOnLevel + 1) * numLevels)

variableVerticesInner = []
offsetListIndex = 0
for level in range(0 + 1, numLevels + 1):
    nextVertices = PointsInCircum(radius + offset,level * stepBetweenLevels,numPtsOnLevel)
    for i in range(len(nextVertices) - 1):
        factor = offsetList[offsetListIndex]
        nextVertices[i][0] *= factor
        nextVertices[i][1] *= factor
        if (i == 0):
            nextVertices[-1] = nextVertices[0]
            print(nextVertices[0])
            print(nextVertices[-1])
        offsetListIndex += 1
    variableVerticesInner += [nextVertices]

allVerticesInner = mouthVerticesInner + variableVerticesInner + topVerticesInner

# allMouthVertices = mouthVerticesInner + mouthVerticesMid + mouthVerticesOuter

print("Successfully generated points and a unique tensor!")


35
[1.3402109439807153, 0.0, 1.0]
[1.3402109439807153, 0.0, 1.0]
[2.820777575307549, 0.0, 2.0]
[2.820777575307549, 0.0, 2.0]
[1.713586485345742, 0.0, 3.0]
[1.713586485345742, 0.0, 3.0]
[1.6604466733162015, 0.0, 4.0]
[1.6604466733162015, 0.0, 4.0]
[2.952266079744712, 0.0, 5.0]
[2.952266079744712, 0.0, 5.0]
Successfully generated points and a unique tensor!


Preview Geometry

In [91]:
curve = BSpline.Curve()
curve.degree= 3
curve.ctrlpts = allVerticesInner[0]
curve.knotvector = utilities.generate_knot_vector(curve.degree, curve.ctrlpts_size)

# print(curve.knotvector)
# operations.refine_knotvector(curve, [1])
# print(curve.knotvector)

utilities.generate_knot_vector()

tangent_vector = utilities.vector_generate(curve.ctrlpts[0], curve.ctrlpts[-1])  # Calculate the vector between start and end points
curve.ctrlpts[1] = tuple(utilities.vector_add(curve.ctrlpts[0], tangent_vector))  # Set the modified control point 1
curve.ctrlpts[-2] = tuple(utilities.vector_add(curve.ctrlpts[-1], tangent_vector))  # Set the modified control point 2

curve.vis = vis.VisCurve2D(ctrlpts=True, legend=False)
print("Generating visualization, please wait...")
curve.render(colormap=cm.terrain)

AttributeError: module 'geomdl.utilities' has no attribute 'vector_generate'

In [80]:
surf = BSpline.Surface()
surf.degree_u = 3
surf.degree_v = 3
surf.ctrlpts2d = allVerticesInner
surf.knotvector_u = utilities.generate_knot_vector(surf.degree_u, surf.ctrlpts_size_u)
surf.knotvector_v = utilities.generate_knot_vector(surf.degree_v, surf.ctrlpts_size_v)

print(surf.knotvector_v)
# surf.knotvector_v = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.91, 1.0]

surf.vis = vis.VisSurface(ctrlpts=True, legend=False)
print("Generating visualization, please wait...")
surf.render(colormap=cm.terrain)

[0.0, 0.0, 0.0, 0.0, 0.25, 0.5, 0.75, 1.0, 1.0, 1.0, 1.0]
Generating visualization, please wait...


Save Geometry

In [14]:
from geomdl import abstract

vecName = ".\\tensor\\example_tensor.txt"
stlName = ".\\stl\\example_stl.stl"


with open(vecName, "w") as output:
    output.write(str(offsetList))
print("Data successfully exported!\n...")

exchange.export_stl(sphere, stlName)
print("Surface .stl successfully exported!")

Data successfully exported!
...
Surface .stl successfully exported!
